# Computing Point-in-Time Residual Returns

In this notebook, we will use rolling regressions to compute beta-adjusted (residual) returns for a set of technology stocks in a point-in-time manner suitable for backtesting or live trading.
Residual returns, often referred to as alphas, represent the component of a stock’s return that cannot be explained by its exposure to a benchmark (e.g., an industry ETF). These values are essential for identifying idiosyncratic performance and building market-neutral trading strategies.

We will be performing this analysis in the following few steps
- Downloading of historical data
- Estimate the rolling betas through the use of a lookback window
- Compute the daily residual (Alpha) returns
- Analyse volatility (original vs residual returns)
- Comparing correlations (original vs residual returns)
- Performance ratios (Information and Sharpe)

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf

##### 1️⃣ Data Collection — Download Historical Prices

We begin by downloading daily close prices for the following tickers from Yahoo Finance, starting from 2016-01-01:


| Stock | Description                                                |
| :---- | :--------------------------------------------------------- |
| FB    | Meta Platforms (Facebook)                                  |
| AAPL  | Apple Inc.                                                 |
| AMZN  | Amazon.com Inc.                                            |
| NFLX  | Netflix Inc.                                               |
| GOOGL | Alphabet Inc.                                              |
| QQQ   | Invesco QQQ Trust (NASDAQ 100 ETF) — used as the benchmark |

From these prices, compute daily returns using the adjusted close data:

$$ R_t = \frac{P_t}{P_{t-1}} - 1 $$

where:
- $ P_t $: Adjusted close price at time $ t $
- $ P_{t-1} $: Adjusted close price at time $ t-1 $

In [2]:
tickers = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'QQQ']
data = yf.download(tickers, start='2016-01-01')['Close'] # Retrieving the closing prices
returns = data.pct_change() # Compute the percentage change to get daily returns
returns.head()

/var/folders/5v/9xrz7nmd29s2z029pvbqs8r00000gn/T/ipykernel_6091/1206028535.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start='2016-01-01')['Close'] # Retrieving the closing prices
[*********************100%***********************]  6 of 6 completed


Ticker,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
Date,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,-0.025059,-0.005024,NaN,0.002752,-0.020917,-0.001735
2016-01-06,-0.019569,-0.001799,NaN,-0.002889,0.093071,-0.009606
2016-01-07,-0.042205,-0.039058,NaN,-0.024140,-0.026513,-0.031314
2016-01-08,0.005288,-0.001464,NaN,-0.013617,-0.027671,-0.008201


##### 2️⃣ Estimating Rolling Betas (252-Day Lookback)

Next, we estimate the beta of each stock relative to **QQQ**, using a **rolling 252-day window** (approximately one trading year). This approach ensures our beta estimates are **point-in-time** and **avoid lookahead bias**. The Formula for Beta is as follows:

$$
\beta = \frac{\text{Cov}(R_{\text{stock}}, R_{\text{benchmark}})}{\text{Var}(R_{\text{benchmark}})}
$$

where:  
- $ R_{\text{stock}} $ = returns of the stock  
- $ R_{\text{benchmark}} $ = returns of QQQ  

This measures the **sensitivity** of the stock’s return to movements in the benchmark.

For reference, the related concept of **correlation** between a stock and the benchmark is given by:

$$
\text{Corr}(R_{\text{stock}}, R_{\text{benchmark}}) = \frac{\text{Cov}(R_{\text{stock}}, R_{\text{benchmark}})}{\sigma_{R_{\text{stock}}} \cdot \sigma_{R_{\text{benchmark}}}}
$$

where:  
- $ \text{Cov} $ is the covariance,  
- $ \sigma_{R_{\text{stock}}} $ and $ \sigma_{R_{\text{benchmark}}} $ are the standard deviations of returns.

> **Correlation** measures **co-movement**, while **beta** measures **sensitivity and magnitude**.

In [3]:
# Compute the rolling 252-day beta for each stock relative to QQQ
benchmark = returns['QQQ'] # Isolate the benchmark returns
corr = returns.rolling(window=252).corr(benchmark) # Rolling correlation with benchmark
vol = returns.rolling(window=252).std() # Rolling volatility of each stock
beta = corr.multiply(vol, axis=0).divide(vol['QQQ'], axis=0) # Beta calculation
beta.tail()

Ticker,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
Date,,,,,,
2026-01-13,0.995206,1.132034,NaN,0.911120,0.729323,1.0
2026-01-14,0.993234,1.134738,NaN,0.909153,0.732234,1.0
2026-01-15,0.992623,1.134821,NaN,0.908199,0.731382,1.0
2026-01-16,0.994079,1.134631,NaN,0.904966,0.728344,1.0
2026-01-20,0.994635,1.137916,NaN,0.906370,0.725146,1.0


##### 3️⃣ Computing Daily Residual (Alpha) Returns

Once we have rolling betas, we can decompose the stock’s return into **expected** and **residual (alpha)** components. Recall that the expected return from benchmark exposure is expressed as follow:
$$
E[R_{\text{stock},t}] = \beta_{\text{stock},t} \cdot R_{\text{benchmark},t}
$$

Given our initial expression for expected returns of a stock being as follow:

$$
R_{\text{stock},t} = \beta_{\text{stock},t} \cdot R_{\text{benchmark},t} + \alpha_{\text{stock},t}
$$

We can rearrange the expression above to get our residual (Alpha) return:
$$
\alpha_{\text{stock},t} = R_{\text{stock},t} - \beta_{\text{stock},t} \cdot R_{\text{benchmark},t}
$$

where:  
- $ R_{\text{stock},t} $ = actual stock return on day $ t $  
- $ \beta_{\text{stock},t} $ = rolling beta (point-in-time)  
- $ R_{\text{benchmark},t} $ = benchmark (QQQ) return on day $ t $  
- $ \alpha_{\text{stock},t} $ = **residual return** (stock-specific performance, *alpha*)  

> These **residuals** represent **stock-specific outperformance or underperformance**, after adjusting for systematic exposure to the market (QQQ).

In [4]:
# Compute the residual returns of our stock
residuals = returns.subtract(beta.multiply(benchmark, axis=0), axis=0)
residuals.tail()

Ticker,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
Date,,,,,,
2026-01-13,0.004550,-0.014023,NaN,0.013736,0.011259,-7.589415e-18
2026-01-14,0.006435,-0.012404,NaN,0.009325,-0.011775,-5.551115e-17
2026-01-15,-0.010305,0.002381,NaN,-0.012380,-0.008279,1.821460e-17
2026-01-16,-0.009548,0.004896,NaN,-0.007597,0.000041,-4.445229e-18
2026-01-20,-0.013422,-0.009780,NaN,-0.004985,0.006998,-1.075529e-16


##### 4️⃣ Analyzing Volatility: Original vs. Residual Returns
Let us compare the volatility (standard deviation) of the raw and residual returns. You should observe that:

> ⚙️ Residual returns typically have lower volatility than original returns, since the benchmark-driven (systematic) risk component has been removed.

This demonstrates how much of each stock’s risk was tied to the overall tech sector (via QQQ).

In [5]:
# Compute and compare the volatilities of the residual returns and the original returns
vol = {}
vol['original'] = returns.std()*np.sqrt(252)
vol['residual'] = residuals.std()*np.sqrt(252) 
vol = pd.DataFrame(vol)
vol

,original,residual
Ticker,,
AAPL,0.290202,1.698946e-01
AMZN,0.327948,2.040368e-01
FB,0.049516,NaN
GOOGL,0.287971,1.822493e-01
NFLX,0.418765,3.276162e-01
QQQ,0.223104,1.107556e-15


As shown above, we can see that the residual returns generally have much lower volatility, as we have taken out the 'beta' component which drives much of the stock returns

##### 5️⃣ Comparing Correlations: Original vs. Residual Returns

Now, let us compute and compare the pairwise correlations of:
- Original stock returns
- Residual (alpha) returns

Expected observation:
> 📉 The correlations between stocks decrease after adjusting for benchmark exposure.

This shows that much of the co-movement among tech stocks is due to shared market/industry factors rather than unique, idiosyncratic behavior.

In [6]:
returns.corr()

Ticker,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,0.573097,0.312062,0.609954,0.430997,0.806418
AMZN,0.573097,1.000000,0.452223,0.635905,0.523804,0.767837
FB,0.312062,0.452223,1.000000,0.344560,0.115576,0.662611
GOOGL,0.609954,0.635905,0.344560,1.000000,0.441470,0.785516
NFLX,0.430997,0.523804,0.115576,0.441470,1.000000,0.585970
QQQ,0.806418,0.767837,0.662611,0.785516,0.585970,1.000000


In [7]:
residuals.corr()

Ticker,AAPL,AMZN,FB,GOOGL,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,-0.099670,NaN,-0.060529,-0.092909,-0.016202
AMZN,-0.099670,1.000000,NaN,0.060655,0.131819,0.007374
FB,NaN,NaN,NaN,NaN,NaN,NaN
GOOGL,-0.060529,0.060655,NaN,1.000000,-0.063591,-0.016655
NFLX,-0.092909,0.131819,NaN,-0.063591,1.000000,-0.017564
QQQ,-0.016202,0.007374,NaN,-0.016655,-0.017564,1.000000


As shown in the above correlation matrix for original returns and residuals (Alpha), the pairwise correlations of the residual returns are generally much lower between stocks. This is because we have taken out one of the major common forces or tides moving these stocks. Another observation is the large drop in correlations with QQQ. The original returns are 0.6 to 0.8 correlated with the QQQ, but the residual returns are almost near 0 correlated with the benchmark QQQ. 

##### 6️⃣ Performance Ratios: Information Ratio vs. Sharpe Ratio

To evaluate the **risk-adjusted performance** of each stock and its **alpha component**, we compute two key metrics:

1. **Information Ratio (IR)**

$$
\text{IR} = \frac{\text{Mean}(\alpha_{\text{stock}})}{\text{Std}(\alpha_{\text{stock}})}
$$

> This measures the **consistency of alpha generation** relative to **benchmark-adjusted risk** (i.e., residual volatility).  
> A **higher IR** indicates **stronger active management skill** or **idiosyncratic return potential**.

2. **Sharpe Ratio**

$$
\text{Sharpe Ratio} = \frac{\text{Mean}(R_{\text{stock}})}{\text{Std}(R_{\text{stock}})}
$$

> This measures **total risk-adjusted performance**, capturing **both systematic and idiosyncratic** sources of return.

---

Interpretation & Comparison

| Metric             | Focus                                |
|--------------------|---------------------------------------|
| **Sharpe Ratio**   | → **Overall return efficiency**       |
| **Information Ratio** | → **Benchmark-adjusted efficiency** (true *active alpha*) |


Typically, **IR < Sharpe Ratio**, because removing benchmark exposure reduces **both mean return and volatility**, isolating the smaller **stock-specific alpha**.

In [8]:
df = {}
df['Information Ratio'] = residuals.mean() / residuals.std() * np.sqrt(252)
df['Sharpe Ratio'] = returns.mean() / returns.std() * np.sqrt(252)
df = pd.DataFrame(df)
df

,Information Ratio,Sharpe Ratio
Ticker,,
AAPL,0.327455,0.950112
AMZN,0.048105,0.766786
FB,NaN,2.211243
GOOGL,0.303154,0.887776
NFLX,0.208526,0.707516
QQQ,0.594307,0.911324
